#Databricks Mount para o AWS S3 e importar/exportar dados

###1. Criar AWS Access Key e Secret Key para o Databricks

####1.1. Procure por "IAM" no AWS Search Bar e clique em IAM.

In [0]:
displayHTML("<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_1_iam.png")

<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_1_iam.png

####1.2. Clique em Users no menu Access Management no lado esquerdo da página.

In [0]:
displayHTML("<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_2_users.png")

<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_2_users.png

####1.3. Clique no botão azul "Add Users".

In [0]:
displayHTML("<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_3_specify_user.png")

<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_3_specify_user.png

####1.4. Vá para a seção "Permissions" e selecione "Attach existing policies directly". Digite "AmazonS3FullAccess" no search bar e selecione a caixa no search result.

In [0]:
displayHTML("<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_5_policies.png")

<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_5_policies.png

####1.5. Pular a seção "Tag" e clicar em "Create user".

In [0]:
displayHTML("<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_6_tags.png")

<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_6_tags.png

####1.6. Clique no User criado e em seguida na tab "Security credentials".

In [0]:
displayHTML("<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_6_security_credentials.png")

<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_6_security_credentials.png

####1.7. Clique em "Create access key"

In [0]:
displayHTML("<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_7_create_access_key.png")

<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_7_create_access_key.png

####1.8. Selecione a opção "Application running outside AWS" e em seguida no botão "Next". Na página seguinte, clique em "Create access key"

In [0]:
displayHTML("<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_8_access_key_best_practices.png")

<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_8_access_key_best_practices.png

####1.9. Cique no botão "Download .csv file".

In [0]:
displayHTML("<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_9_download_csv.png")

<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/1_9_download_csv.png

###2. Fazer o upload do arquivo contendo a credencial do AWS para Databricks

####2.1. Clique no ícone Data e em seguida clique em Create Table. Obs. você precisará iniciar um cluster antes de criar uma tabela.

In [0]:
displayHTML("<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/2_1_create_table.png")

<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/2_1_create_table.png

####2.2. Clique em "Drop files to upload", ou clique para procurar o arquivo e faça upload do arquivo baixado da AWS no passo 1.

In [0]:
displayHTML("<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/2_2_drop_files.png")

<img src=https://raw.githubusercontent.com/felipeyoshi/americanas_bigdata/main/images/2_2_drop_files.png

###3. Fazer o mount do S3 Bucket para o Databricks

####3.1. Listar as tabelas do FileStore através do código abaixo.

In [0]:
# Listar os objetos da pasta tables
dbutils.fs.ls('/FileStore/tables')

Out[1]: [FileInfo(path='dbfs:/FileStore/tables/3fbeab8b_1b87_4f2b_9c9d_c6d2b5f02564.csv', name='3fbeab8b_1b87_4f2b_9c9d_c6d2b5f02564.csv', size=254832, modificationTime=1678625841000),
 FileInfo(path='dbfs:/FileStore/tables/54f2b05b_1fe0_44f9_ae75_5e84fb3da81f-1.csv', name='54f2b05b_1fe0_44f9_ae75_5e84fb3da81f-1.csv', size=15590616, modificationTime=1678624125000),
 FileInfo(path='dbfs:/FileStore/tables/54f2b05b_1fe0_44f9_ae75_5e84fb3da81f.csv', name='54f2b05b_1fe0_44f9_ae75_5e84fb3da81f.csv', size=15590616, modificationTime=1678622740000),
 FileInfo(path='dbfs:/FileStore/tables/BB_agency_axim_2.csv', name='BB_agency_axim_2.csv', size=611828, modificationTime=1657214142000),
 FileInfo(path='dbfs:/FileStore/tables/BB_bgov_market_axim_2.csv', name='BB_bgov_market_axim_2.csv', size=209223, modificationTime=1657214139000),
 FileInfo(path='dbfs:/FileStore/tables/BB_fusion_axim_2.csv', name='BB_fusion_axim_2.csv', size=1637570, modificationTime=1657214138000),
 FileInfo(path='dbfs:/FileStore

####3.2. Fazer o mount do S3 bucket.

In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

# Definir o tipo de arquivo
file_type = 'csv'
# Setar que o arquivo contém a pirmeira linha como schema
first_row_is_header = True
# Setar o delimitador do arquivo csv
delimiter = ','
# Ler o arquivo csv e carregar como spark dataframe
aws_keys_df = (spark.read.format(file_type)
               .option('header', first_row_is_header)
               .option('sep', delimiter)
               .load('/FileStore/tables/tutorial_demo.csv')
              )

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe='')

In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = 'databricks-felipeyoshimoto'
# Mount name for the bucket
MOUNT_NAME = '/mnt/databricks-felipeyoshimoto'
# Source url
SOURCE_URL = 's3n://{0}:{1}@{2}'.format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[19]: True

In [0]:
# Check if the AWS S3 bucket was mounted successfully
display(dbutils.fs.ls('/mnt/databricks-felipeyoshimoto/'))

path,name,size,modificationTime
dbfs:/mnt/databricks-felipeyoshimoto/bronze/,bronze/,0,0


In [0]:
#Descomente esse código para fazer Unmount do S3
#dbutils.fs.unmount('/mnt/databricks-felipeyoshimoto/')

/mnt/databricks-felipeyoshimoto/ has been unmounted.
Out[18]: True